In [7]:
import cv2
import os

import numpy as np

path=r'E:/tupian/'
#由于我们的数据集图片大小不一致，所以需要resize成统一大小 这里resize成100x100x3
w=100
h=100
c=3
#data 对应图片, label 是标签，roses 0,daisy 1,sunflowers 2,tulips 3,dandelion 4.
def read_img(path):
    imgs=[]
    labels=[]
    cate=[path+x for x in os.listdir(path) if os.path.isdir(path+x)]
    for idx,i in enumerate(cate):
        for j in os.listdir(i):
            im = cv2.imread(i+'/'+j)
            img = cv2.resize(im, (w, h))/255.
            imgs.append(img)
            labels.append(idx)
    return np.asarray(imgs,np.float32),np.asarray(labels,np.int32)
data,label=read_img(path)
#打乱顺序
num_example=data.shape[0] # data.shape是(3029, 100, 100, 3)
arr=np.arange(num_example)# 创建等差数组 0，1，...,3028
np.random.shuffle(arr)# 打乱顺序
data=data[arr]
label=label[arr]
print(label)


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 1
 1 0 1 1 1 0 1 1 1 1 0 1 0 1]


In [8]:
train_images=data
train_labels=label

In [15]:
len(train_images)

88

In [22]:
from PIL import Image

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import time

train_images = train_images.reshape(train_images.shape[0], 100, 100,3).astype('float32')
print(train_labels[0])
plt.imshow(train_images[0, :, :], cmap='gray')
plt.show()

class Dense(layers.Layer):
    def __init__(self, input_dim, units):
        super(Dense,self).__init__()
        # initializer = tf.initializers.glorot_uniform()
        initializer = tf.initializers.glorot_normal()
        self.w = tf.Variable(initial_value=initializer(shape=(input_dim,units),dtype=tf.float32),trainable=True)
        self.b = tf.Variable(initial_value=tf.zeros(shape=(1,units),dtype=tf.float32),trainable=True)#节点的偏置也是行向量 才可以正常计算 即对堆叠的batch 都是加载单个batch内
    @tf.function
    def call(self,x,training=True):
        if training == True:
            y = tf.matmul(x,self.w)+self.b
            return y
        else:
            y = tf.matmul(x,self.w)+self.b
            return y 
class Discriminator(tf.keras.Model):
    def __init__(self):
        super(Discriminator,self).__init__()
        self.dense1 = Dense(100*100*3,128)
        self.dense2= Dense(128,32)
        self.dense3 = Dense(32,1)
    @tf.function
    def call(self,x,training=True):
        """
        batch*dim+batch*10 在index_1维度组合 其余维度不变
        """
        x = tf.reshape(x,[-1,30000]) #reshape 不改变原始的元素顺序 这很重要 防止变形时变成转置 忽略batch大小  只关注后面的维度一致
        if training == True:
            l1_out = tf.nn.relu(self.dense1(x,training))
            l2_out = tf.nn.relu(self.dense2(l1_out,training))
            l3_out = tf.nn.sigmoid(self.dense3(l2_out,training))
            return l3_out
        else:
            l1_out = tf.nn.relu(self.dense1(x,training))
            l2_out = tf.nn.relu(self.dense2(l1_out,training))
            l3_out = self.dense3(l2_out,training) #!!!!!! WGAN  判别器不能加sigmoid 
            # l3_out = tf.nn.sigmoid(self.dense3(l2_out,training))
            return l3_out
    @tf.function       
    def to_clip(self):
        for weight in self.trainable_variables:
            weight.assign(tf.clip_by_value(weight,clip_value_min=-0.01,clip_value_max=0.01))
d = Discriminator()
x = train_images[0:2, :, :]
print(d(x,training=False))#行向量统一输入  而batch是行向量在列方向堆叠后的矩阵 
print(len(d.trainable_variables))

class Generator(tf.keras.Model):
    def __init__(self):
        super(Generator,self).__init__()
        self.dense1 = Dense(100,32)
        self.dense2 = Dense(32,128)
        self.dense3 = Dense(128,30000)
    @tf.function
    def call(self,x,training=True):
        if training == True:
            l1_out = tf.nn.relu(self.dense1(x,training))
            l2_out = tf.nn.relu(self.dense2(l1_out,training))
            l3 = tf.nn.sigmoid(self.dense3(l2_out,training))
        else:
            l1_out = tf.nn.relu(self.dense1(x,training))
            l2_out = tf.nn.relu(self.dense2(l1_out,training))
            l3 = tf.nn.sigmoid(self.dense3(l2_out,training))
        l3_out = tf.reshape(l3,[-1,100,100,3])
        return l3_out

g = Generator()
z = tf.random.normal((1,100))
image = g(z,training=False)
#plt.imshow(tf.reshape(image,(100,100,3)), cmap='gray')
#plt.show()

print(d(image,training=False))

def d_loss(real_output, fake_output):
    total_loss = -tf.reduce_mean(real_output)+tf.reduce_mean(fake_output)#用batch 均值逼近期望 然后依据公式 max  所以取反  -E(real)+E(fake)  做min
    return total_loss
def g_loss(fake_output):
    return -tf.reduce_mean(fake_output)

generator_optimizer = tf.keras.optimizers.RMSprop(learning_rate=3e-4,epsilon=1e-10)
discriminator_optimizer = tf.keras.optimizers.RMSprop(learning_rate=3e-4,epsilon=1e-10)

EPOCHS = 400
BATCH_SIZE = 128
z_dim = 100
num_examples_to_generate = 100
seed = tf.random.normal([num_examples_to_generate, z_dim],mean=0.0,stddev=1.0)
# seed = tf.random.uniform([num_examples_to_generate, z_dim],-1.0,1.0)

@tf.function
def D_train_step(images,labels):
    z = tf.random.normal([images.shape[0], z_dim],mean=0.0,stddev=1.0)
    # z = tf.random.uniform([images.shape[0], z_dim],-1.0,1.0)
    with tf.GradientTape() as disc_tape:
        generated_images = g(z,training=True)
        real_output = d(images,training=True)
        fake_output = d(generated_images,training=True)
        disc_loss = d_loss(real_output,fake_output)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, d.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, d.trainable_variables))
    d.to_clip()

@tf.function
def G_train_step(images,labels):
    z = tf.random.normal([images.shape[0],z_dim],mean=0.0,stddev=1.0)
    # z = tf.random.uniform([images.shape[0], z_dim],-1.0,1.0)
    with tf.GradientTape() as gen_tape:
        generated_images = g(z,training=True)
        fake_output = d(generated_images ,training=True)
        gen_loss = g_loss(fake_output)
    gradients_of_generator = gen_tape.gradient(gen_loss, g.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, g.trainable_variables))

def train(train_images,train_labels,epochs):
    break_flag = 0
    index = list(range(train_images.shape[0]))
    np.random.shuffle(index)
    train_images = train_images[index]
    train_labels = train_labels[index]
    images_batches = iter(tf.data.Dataset.from_tensor_slices(train_images).batch(BATCH_SIZE))
    labels_batches = iter(tf.data.Dataset.from_tensor_slices(train_labels).batch(BATCH_SIZE))
    for epoch in range(epochs):
        start = time.time()
        while True:
            for i in range(5):
                try:
                    x_real_bacth = next(images_batches)
                    y_label_bacth = next(labels_batches)
                    D_train_step(x_real_bacth,y_label_bacth)
                except StopIteration:
                    del images_batches
                    del labels_batches
                    np.random.shuffle(index)
                    train_images = train_images[index]
                    train_labels = train_labels[index]
                    images_batches = iter(tf.data.Dataset.from_tensor_slices(train_images).batch(BATCH_SIZE))
                    labels_batches = iter(tf.data.Dataset.from_tensor_slices(train_labels).batch(BATCH_SIZE))
                    break_flag = 1
                    break
            if break_flag == 0: # 判别器训练5次 然后进行一次生成器
                G_train_step(x_real_bacth,y_label_bacth)
            else:
                break_flag = 0
                break
        generate_and_save_images(g,epoch + 1,seed)
        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input,training=False)
    #plt.figure(figsize=(10,10))
    predictions=np.array(predictions)*255.0
    for i in range(predictions.shape[0]):
        #print(predictions[i])
        newtqm=Image.fromarray(predictions[i].astype(np.uint8))
        newtqm.save('./WGAN/'+str(epoch) + "_" + str(i) + ".png")
        #plt.subplot(10,10,i+1)
        #plt.imshow(tf.reshape(predictions[i,:],shape=(100,100,3))*255.0, cmap='gray')
        #plt.axis('off')
    
    #plt.savefig('./WGAN/image_at_epoch_{:04d}.png'.format(epoch))
    #plt.close()

print(time)
train(train_images,train_labels,EPOCHS)